In [1]:
# Loading Dependencies
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.metrics import confusion_matrix
import itertools
import os
import matplotlib.pyplot as plt
from tensorflow.keras.applications.inception_v3 import InceptionV3
from glob import glob
from tensorflow.keras import layers
import time

In [ ]:

# Importing the Model
# my_model = load_model("C:/Users/tiwar/Desktop/Project_transfer_learning_model_20_epochs.h5")

In [3]:
from PIL import Image
import cv2
import numpy as np
from skimage import transform

# Function to load and preprocess the image
def load(filename):
    image_classifier = 0
    display_image = filename
    # display_image = cv2.cvtColor(display_image, cv2.COLOR_BGR2RGB)

    if display_image.mean() > 125:
        box_image = display_image[50:290, :]
        image_classifier = 1
    else:
        box_image = display_image[30:430, :]
        image_classifier = 2

    prediction_image = Image.fromarray(box_image)
    prediction_image = np.array(prediction_image).astype('float32') / 255
    prediction_image = transform.resize(prediction_image, (224, 224, 3))
    prediction_image = np.expand_dims(prediction_image, axis=0)

    return prediction_image, box_image, display_image, image_classifier


In [2]:
def make_box(input_image):
    # Preprocess the image
    gray_img = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    # Applying 7x7 Gaussian Blur
    blurred = cv2.GaussianBlur(gray_img, (7, 7), 0)

    # Applying threshold
    threshold = cv2.threshold(
        blurred, int(input_image.mean()) + 80, 255, cv2.THRESH_BINARY
    )[1]

    # Apply the Component analysis function
    analysis = cv2.connectedComponentsWithStats(threshold, 4, cv2.CV_32S)
    (totalLabels, label_ids, values, centroid) = analysis

    # Initialize a new image to store all the output components
    output = np.zeros(gray_img.shape, dtype="uint8")
    count = 0
    rects = []
        # Loop through each component
    for i in range(1, totalLabels):
        area = values[i, cv2.CC_STAT_AREA]
        if (area > 30) and (area < 300):
            componentMask = (label_ids == i).astype("uint8") * 255
            output = cv2.bitwise_or(output, componentMask)
            x1 = values[i, cv2.CC_STAT_LEFT]
            y1 = values[i, cv2.CC_STAT_TOP]
            w = values[i, cv2.CC_STAT_WIDTH]
            h = values[i, cv2.CC_STAT_HEIGHT]
            pt1 = (x1 - 40, y1 - 40)
            pt2 = (x1 + w + 40, y1 + h + 40)
            rects.append([pt1, pt2])
            count = count + 1

    for i in rects:
        cv2.rectangle(input_image, i[0], i[1], (0, 255, 0), 4)

    return input_image


In [ ]:
source_video = "C:/Users/tiwar/Desktop/videos/24.avi"
cap = cv2.VideoCapture(source_video)
count = 0

while cap.isOpened():
    count += 1
    frame_count = f"Frame {count}"
    ret, frame = cap.read()
    actual_image = frame.copy()
    actual_image = cv2.resize(actual_image, (678, 384))
    cv2.putText(actual_image, 'Actual_Video', org=(20, 25), fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
                fontScale=0.5, color=(0, 0, 255), thickness=1)
    cv2.putText(actual_image, frame_count, org=(200, 25), fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
                fontScale=0.5, color=(0, 0, 255), thickness=1)

    if ret == False:
        break

    prediction_image, box_image, display_image, image_classifier = load(frame)
    model_prediction = my_model.predict(prediction_image)

    if model_prediction < 0.5:
        box_image = make_box(box_image)
        if image_classifier == 1:
            display_image[50:290, :] = box_image
        elif image_classifier == 2:
            display_image[30:430, :] = box_image

        display_image = cv2.resize(display_image, (678, 384))
        cv2.putText(display_image, 'A06 Test analyzed_Video', org=(20, 25), fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
                    fontScale=0.5, color=(0, 0, 255), thickness=1)
        cv2.putText(display_image, frame_count, org=(200, 25), fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
                    fontScale=0.5, color=(0, 0, 255), thickness=1)
        cv2.putText(display_image, 'NG_Scratch Detected !!', org=(20, 350), fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
                    fontScale=1, color=(0, 0, 255), thickness=2)

    else:
        display_image = cv2.resize(display_image, (678, 384))
        cv2.putText(display_image, 'A06 Test analyzed_Video', org=(20, 25), fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
                    fontScale=0.5, color=(0, 0, 255), thickness=1)
        cv2.putText(display_image, frame_count, org=(200, 25), fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
                    fontScale=0.5, color=(0, 0, 255), thickness=1)
        cv2.putText(display_image, 'OK_No Scratches Found', org=(20, 350), fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
                    fontScale=1, color=(255, 0, 0), thickness=2)

    concatenated_image = np.concatenate((actual_image, display_image), axis=1)
    cv2.imshow("A06 Scratch Detection Model", concatenated_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
